In [ ]:
%pip install --upgrade transformers

In [1]:
%pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/rocm7.1

Looking in indexes: https://download.pytorch.org/whl/nightly/rocm7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 16.1 MB/s  0:00:00 14.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 GB 25.7 MB/s  0:02:34 eta 0:00:010:00:05
  Attempting uninstall: torch
    Found existing installation: torch 2.11.0.dev20251230+rocm7.1
    Uninstalling torch-2.11.0.dev20251230+rocm7.1:━━━━ 0/3 [torch]
      Successfully uninstalled torch-2.11.0.dev20251230+rocm7.1m0/3 [torch]
  Attempting uninstall: torchvision━━━━━━━━━━━━━━━━━━━ 0/3 [torch]
    Found existing installation: torchvision 0.25.0.dev20251231+rocm7.1rch]
    Uninstalling torchvision-0.25.0.dev20251231+rocm7.1:0/3 [torch]
      Successfully uninstalled torchvision-0.25.0.dev20251231+rocm7.1torch]
  Attempting uninstall: torchaudio╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [torchvision]
    Found existing installation: torchaudio 2.10.0.dev20251231+rocm7.1━━━━━━━━━━━━━━━━━ 1/3 [torchvision]
    Uninstalling torchaudio-2.

In [ ]:
%pip install transformers==5.0.0rc1

In [ ]:
%pip install python-dotenv

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")

print(f"HF_TOKEN: {HF_TOKEN[0]}")

HF_TOKEN: h


In [5]:
#!export HIP_LAUNCH_BLOCKING=1

In [5]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["HSA_OVERRIDE_GFX_VERSION"] = "11.0.0"
#os.environ["HIP_LAUNCH_BLOCKING"]="1"
#os.environ["AMD_LOG_LEVEL"]="3"
print(os.environ.get("HIP_LAUNCH_BLOCKING", ""))
print(os.environ.get("AMD_LOG_LEVEL", ""))


In [6]:
import os
import torch
import transformers
from transformers import TokenizersBackend

print(f"PyTorch Version: {torch.__version__}")
print(f"ROCm Version:    {torch.version.hip}")
print(f"transformers Version:    {transformers.__version__}")

/home/tiry/tairy/notebooks/venv-rocm/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch Version: 2.11.0.dev20260105+rocm7.1
ROCm Version:    7.1.52802
transformers Version:    5.0.0rc1


In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from transformers import FineGrainedFP8Config
from transformers import Mistral3ForConditionalGeneration, Mistral3Config, PixtralVisionConfig, MistralConfig

#  4-Bit Quantization Config 
bnb4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

bnb8_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

# use for 8B model
q8_config = FineGrainedFP8Config()


def print_model_specs(model):
    # 1. Get the number of parameters
    # set only_trainable=False to include frozen weights (e.g. if using LoRA)
    total_params = model.num_parameters(only_trainable=False)
    
    # 2. Get the memory footprint (size of weights in VRAM/RAM)
    # This method is specific to Hugging Face models
    memory_bytes = model.get_memory_footprint()
    
    # Format for readability
    print(f"Model Parameters: {total_params / 1_000_000_000:.2f} Billion")
    print(f"Memory Footprint: {memory_bytes / 1024**3:.2f} GB")

/opt/amdgpu/share/libdrm/amdgpu.ids: No such file or directory


In [8]:
import time
import torch
from threading import Thread
from transformers import TextIteratorStreamer, AutoTokenizer, AutoModelForCausalLM

import torch
import gc

# Delete previous model variables if they exist
# (Change 'model' to whatever variable name you used before)
try:
    del model
except NameError:
    pass

# Force Python garbage collection
gc.collect()

# Force PyTorch to release cache
torch.cuda.empty_cache()

#max_memory_mapping = {0: "8GiB", "cpu": "48GiB"}


# --- 1. Setup Device ---
# Strix Halo (8060S) works best with float16 on ROCm 6.2+
if torch.cuda.is_available():
    device = "cuda"
    dtype = torch.float16  
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    dtype = torch.float32
    print("⚠️  GPU Not Detected. CPU mode.")


model_id = "mistralai/Ministral-3-3B-Instruct-2512"
#model_id = "mistralai/Ministral-3-8B-Instruct-2512"
#model_id = "mistralai/Ministral-3-14B-Instruct-2512-BF16"

print(f"\nLoading {model_id}...")

# TRUST_REMOTE_CODE=True is the key fix here
tokenizer = AutoTokenizer.from_pretrained(
    model_id, 
    trust_remote_code=True
)

model = Mistral3ForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=dtype,    # Standard HF uses torch_dtype, but some custom models prefer dtype
    trust_remote_code=True, # Allow the model to define its own config class
    device_map=device,       # Auto-moves to GPU
#    device_map="auto",       # load model in system ram then move to GPU after quantization
#    quantization_config=bnb4_config,
)

print("Model loaded successfully!")
print(f"Model {model}")

print(f"GPU Memory Used: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

# --- 3. Run Inference ---
messages = [
    {"role": "user", "content": "Tell me a short story."}
]

# Apply Mistral's chat template
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)


# Tokenize and move to device
inputs = tokenizer(prompt, return_tensors="pt").to(device)

#from transformers import AutoProcessor
#processor = AutoProcessor.from_pretrained("model_id")
#inputs = processor(images=None, text=f"{prompt}", return_tensors="pt")
#generate_ids = model.generate(**inputs, max_new_tokens=15)
#print("generated ids={generate_uds}")

generation_kwargs = dict(
    inputs=inputs.input_ids,
    attention_mask=inputs.attention_mask,
     streamer=streamer,
    max_new_tokens=300,    
    do_sample=True,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id
)

print(f"\nPrompt: {messages[0]['content']}")
print(f"\nMistral Prompt: {prompt}")
print("-" * 30)

#print(model.generate(**generation_kwargs))
#print("end generate")

t0 = time.time()
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()

# --- 4. Stream Output ---
generated_text = ""
first_token_received = False
ttft = 0

for new_text in streamer:
    if not first_token_received:
        ttft = time.time() - t0
        first_token_received = True
        print(new_text, end="", flush=True)
    else:
        print(new_text, end="", flush=True)
    generated_text += new_text

t_end = time.time()

# --- 5. Stats ---
total_new_tokens = len(tokenizer.encode(generated_text))
decoding_time = t_end - (t0 + ttft)

print("\n" + "-" * 30)
print(f"Time to First Token: {ttft:.4f} s")
if decoding_time > 0:
    print(f"Generation Speed:    {(total_new_tokens-1)/decoding_time:.2f} tokens/sec")
print(f"Total Tokens:        {total_new_tokens}")
print_model_specs(model)

✅ GPU Detected: AMD Radeon 8060S

Loading mistralai/Ministral-3-3B-Instruct-2512...


/home/tiry/tairy/notebooks/venv-rocm/lib/python3.13/site-packages/transformers/modeling_utils.py:4655: UserWarning: expandable_segments not supported on this platform (Triggered internally at /pytorch/c10/hip/HIPAllocatorConfig.h:40.)
  _ = torch.empty(byte_count // factor, dtype=torch.float16, device=device, requires_grad=False)
Loading weights: 100%|██████████████████████| 822/822 [00:01<00:00, 707.08it/s, Materializing param=model.vision_tower.transformer.layers.23.ffn_norm.weight]


Model loaded successfully!
Model Mistral3ForConditionalGeneration(
  (model): Mistral3Model(
    (vision_tower): PixtralVisionModel(
      (patch_conv): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (ln_pre): PixtralRMSNorm((1024,), eps=1e-05)
      (transformer): PixtralTransformer(
        (layers): ModuleList(
          (0-23): 24 x PixtralAttentionLayer(
            (attention_norm): PixtralRMSNorm((1024,), eps=1e-05)
            (feed_forward): PixtralMLP(
              (gate_proj): Linear(in_features=1024, out_features=4096, bias=False)
              (up_proj): Linear(in_features=1024, out_features=4096, bias=False)
              (down_proj): Linear(in_features=4096, out_features=1024, bias=False)
              (act_fn): SiLUActivation()
            )
            (attention): PixtralAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
  

KeyboardInterrupt: 